In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","12g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.microsoft.sqlserver:mssql-jdbc:9.4.0.jre8") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/worker2/.ivy2/cache
The jars for the packages stored in: /home/worker2/.ivy2/jars
com.microsoft.sqlserver#mssql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-29be41bd-314c-4e96-800d-b8690e6faa01;1.0
	confs: [default]
	found com.microsoft.sqlserver#mssql-jdbc;9.4.0.jre8 in central
:: resolution report :: resolve 252ms :: artifacts dl 4ms
	:: modules in use:
	com.microsoft.sqlserver#mssql-jdbc;9.4.0.jre8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-29be41bd-

[ WARN] Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
CONNECTOR_TYPE='com.microsoft.sqlserver.jdbc.SQLServerDriver'
SQL_USERNAME='tyeson'
SQL_PASSWORD='Canhtoan1509'
SQL_SERVERNAME='tyeson.database.windows.net'
SQL_DBNAME='DW'
CONNECTION_STRING='jdbc:sqlserver://tyeson.database.windows.net:1433;databaseName=DW;'

In [19]:
df = spark.read.option('header','true').csv('/home/worker2/Desktop/thesis/K18/File_K18/sample_crawl_dataset/user.csv',inferSchema=True, escape = '"')

In [20]:
from pyspark.sql.functions import col, split, explode
user = df.select(col('id').alias('userId'), col('username'), col('email'))

In [22]:
user.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_user')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [4]:
list_career = spark.read.format('jdbc')\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_career')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .load()

In [23]:
df = df.select(col('id').alias('userId'), col('career'), split(df.skills,",").alias("skills"))

In [24]:
df = df.select(col('userId'), col('career'), explode(df.skills).alias("skills"))

In [26]:
fact_user_career = df.join(list_career, col('career') == col('careerName'), 'inner').select('userId','careerId', 'skills')

In [9]:
list_competency = spark.read.format('jdbc')\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_competency')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .load()

In [ ]:
fact_user_competency = fact_user_career.join(list_competency, col('skills') == col('competencyName')).select('userId', 'careerId', 'competencyId')

In [30]:
fact_user_competency.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','fact_user')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()